# K-means with H2O

We performed K-means clustering on the longitude and latitude. Inorder to compare results from the k-means performend on manual results. We observed that at the end the top 5 categories stayed almost the same as the results from the manual results.

In [1]:
import h2o
import imp
from h2o.estimators.kmeans import H2OKMeansEstimator

In [2]:
#common category tags to ignore
ignore = ["Restaurants", "Food", "Nightlife"]
#prints the count of unique items in array label
def label_count(labels):
    lab_count = dict()

    for x in labels:
        if x in lab_count:
            lab_count[x] += 1
        else:
            lab_count[x] = 1

    print("Number of Clusters:", len(lab_count))
    for k, v in lab_count.items():
        print(k,"\t",v)
    return len(lab_count)
#prints the top 5 categories for each cluster
#grouped is an dataframe with "clusters" from 0 to n and "categories"
#n is the number of clusters
def print_Categories(grouped, n):

    cat_list = [dict() for x in range(n)]

    for index, row in grouped.iterrows():
        for x in range(n):
            if (row["clusters"] == x):
                cats = row["categories"]
                words = [x.strip() for x in cats.split(",")]
                for word in words:
                    if word not in ignore:
                        if word in cat_list[x]:
                            cat_list[x][word] += 1
                        else:
                            cat_list[x][word] = 1
                break


    for x in range(n):
        print("\n\n", x , "category dictionary")
        sorted_x = sorted(cat_list[x].items(), key=lambda kv: kv[1])
        sorted_x = list(reversed(sorted_x))
        for y in range(5):
            if(y < len(sorted_x)):
                print(sorted_x[y])
    return

In [3]:
# Start a local instance of the H2O engine.
h2o.init();

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: java version "1.8.0_111"; Java(TM) SE Runtime Environment (build 1.8.0_111-b14); Java HotSpot(TM) 64-Bit Server VM (build 25.111-b14, mixed mode)
  Starting server from /anaconda3/lib/python3.6/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /var/folders/32/psx5k33x1qj6mhlx0xkdqr0h0000gn/T/tmp1khtunor
  JVM stdout: /var/folders/32/psx5k33x1qj6mhlx0xkdqr0h0000gn/T/tmp1khtunor/h2o_sramini_started_from_python.out
  JVM stderr: /var/folders/32/psx5k33x1qj6mhlx0xkdqr0h0000gn/T/tmp1khtunor/h2o_sramini_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321... successful.


H2O cluster uptime:,03 secs
H2O cluster timezone:,America/Los_Angeles
H2O data parsing timezone:,UTC
H2O cluster version:,3.22.0.2
H2O cluster version age:,19 days
H2O cluster name:,H2O_from_python_sramini_48gzoz
H2O cluster total nodes:,1
H2O cluster free memory:,1.778 Gb
H2O cluster total cores:,4
H2O cluster allowed cores:,4
H2O cluster status:,"accepting new members, healthy"


In [4]:
businesses = h2o.import_file(path="cleaned_businesses.csv")
businesses.describe()

Parse progress: |█████████████████████████████████████████████████████████| 100%
Rows:58365
Cols:7




,C1,business_id,name,categories,latitude,longitude,city
type,int,string,enum,enum,real,real,enum
mins,0.0,NaN,,,-71.7539412528,-123.5874264,
mean,93123.0799280391,NaN,,,39.811925331435226,-92.89780926842681,
maxs,188590.0,NaN,,,59.4381813808,115.086769,
sigma,54397.75990221571,NaN,,,5.309953865563447,17.66278107297204,
zeros,1,0,,,0,0,
missing,0,0,0,23,0,0,3
0,0.0,Apn5Q_b6Nz61Tq4XzPdf9A,Minhas Micro Brewery,"Tours, Breweries, Pizza, Restaurants, Food, Hotels & Travel",51.0918130155,-114.031674872,Calgary
1,1.0,AjEbIBw6ZFfln7ePHha9PA,CK'S BBQ & Catering,"Chicken Wings, Burgers, Caterers, Street Vendors, Barbeque, Food Trucks, Food, Restaurants, Event Planning & Services",35.960733700000006,-114.939821,Henderson
2,2.0,O8S5hYJ1SMc8fA4QBtVujA,La Bastringue,"Breakfast & Brunch, Restaurants, French, Sandwiches, Cafes",45.5405031,-73.5993003,Montréal


In [5]:
mask = businesses["city"].ascharacter() == "Toronto"
businesses = businesses[mask, : ]


In [6]:
m = H2OKMeansEstimator(k=8, standardize=False, init="Random")
m.train(training_frame=businesses[:, ["latitude", "longitude"]])

kmeans Model Build progress: |████████████████████████████████████████████| 100%


In [7]:
print(m)

Model Details
H2OKMeansEstimator :  K-means
Model Key:  KMeans_model_python_1544573534259_1


ModelMetricsClustering: kmeans
** Reported on train data. **

MSE: NaN
RMSE: NaN
Total Within Cluster Sum of Square Error: 5.774929386325908
Total Sum of Square Error to Grand Mean: 43.17201388068497
Between Cluster Sum of Square Error: 37.39708449435906
Centroid Statistics: 


,centroid,size,within_cluster_sum_of_squares
,1.0,2888.0,0.5481884
,2.0,890.0,0.6872611
,3.0,621.0,1.5541130
,4.0,493.0,0.7211735
,5.0,436.0,0.6650897
,6.0,1402.0,0.4484816
,7.0,592.0,0.3336805
,8.0,293.0,0.8169416


Scoring History: 


,timestamp,duration,iterations,number_of_reassigned_observations,within_cluster_sum_of_squares
,2018-12-11 16:12:36,0.047 sec,0.0,nan,nan
,2018-12-11 16:12:37,0.173 sec,1.0,7615.0,11.0274128
,2018-12-11 16:12:37,0.217 sec,2.0,1169.0,7.4579160
,2018-12-11 16:12:37,0.239 sec,3.0,536.0,6.8123190
,2018-12-11 16:12:37,0.270 sec,4.0,427.0,6.3532464
,2018-12-11 16:12:37,0.295 sec,5.0,398.0,6.2130933
,2018-12-11 16:12:37,0.321 sec,6.0,398.0,6.0992310
,2018-12-11 16:12:37,0.348 sec,7.0,230.0,5.9977235
,2018-12-11 16:12:37,0.384 sec,8.0,241.0,5.9493553
,2018-12-11 16:12:37,0.399 sec,9.0,239.0,5.8876245


In [12]:
businesses["clusters"] = m.predict(businesses[:, ["latitude", "longitude"]])
dfB = businesses.as_data_frame(use_pandas=True)

kmeans prediction progress: |█████████████████████████████████████████████| 100%


In [14]:
print("\n\nClusters:")
label_count(dfB["clusters"])

#print top 5 categories in each cluster
info = ["categories", "name", "clusters"]
grouped = dfB[info]
grouped = grouped.fillna("")
print_Categories(grouped, 8)



Clusters:
Number of Clusters: 8
5 	 1402
0 	 2888
2 	 621
7 	 293
6 	 592
4 	 436
1 	 890
3 	 493


 0 category dictionary
('Coffee & Tea', 430)
('Bars', 333)
('Sandwiches', 270)
('Canadian (New)', 254)
('Breakfast & Brunch', 230)


 1 category dictionary
('Coffee & Tea', 101)
('Bars', 100)
('Breakfast & Brunch', 84)
('Canadian (New)', 75)
('Pizza', 65)


 2 category dictionary
('Chinese', 170)
('Coffee & Tea', 52)
('Indian', 43)
('Desserts', 42)
('Fast Food', 40)


 3 category dictionary
('Coffee & Tea', 64)
('Pizza', 40)
('Bakeries', 39)
('Specialty Food', 38)
('Canadian (New)', 37)


 4 category dictionary
('Coffee & Tea', 51)
('Japanese', 50)
('Korean', 49)
('Chinese', 48)
('Desserts', 37)


 5 category dictionary
('Bars', 222)
('Coffee & Tea', 185)
('Breakfast & Brunch', 130)
('Italian', 124)
('Canadian (New)', 118)


 6 category dictionary
('Italian', 68)
('Coffee & Tea', 65)
('Bars', 49)
('Japanese', 48)
('Sushi Bars', 47)


 7 category dictionary
('Coffee & Tea', 39)
('Specia